# Homework 3
# Interactive Viz

In [2]:
import pandas as pd
import json
import requests
import os
import folium

## Data loading

In [3]:
df_raw = pd.DataFrame.from_csv('P3_GrantExport.csv', sep=';').reset_index()

print(df_raw.shape)
df_raw.head(2)

(63969, 15)


,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN


## Data cleaning

- We drop the rows with `data not included in P3` as `Approved Amount` and convert the `Approved Amount` column to numeric values.

- We drop the rows without a value for `University`. From the SNSF documentation:
> "This field is only filled if the research is carried out at a Swiss institution, otherwise the field remains blank. In the case of mobility fellowships, it is generally left empty."

- We also drop the rows with `'Nicht zuteilbar - NA'` as `University` (it means NA in german).

- We drop all columns except for `University`,`Approved Amount`,`Institution`

- We convert the amounts in numerical values


In [4]:
def drop_na_rows(df_in):
    # removes lins without an approved amount
    df_out = df_in[df_in['Approved Amount']!='data not included in P3']
    # removes lines missing University
    df_out = df_out[~df_out.University.isnull()]
    df_out = df_out[df_out.University!='Nicht zuteilbar - NA']
    # keeps only interesting columns
    df_out = df_out[['University','Approved Amount','Institution']]
    # converts the approved amount to numeric
    df_out['Approved Amount'] = pd.to_numeric(df_out['Approved Amount'])
    return df_out

df_blue = drop_na_rows(df_raw)

print(df_blue.shape)
amount_total = df_blue['Approved Amount'].sum()
print("total approved amount: %f CHF" % amount_total)
df_blue.head(2)

(48283, 3)
total approved amount: 12835443477.020002 CHF


,University,Approved Amount,Institution
1,Université de Genève - GE,41022.0,Faculté de Psychologie et des Sciences de l'Ed...
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.0,Kommission für das Corpus philosophorum medii ...


Before localizing the universities, let's do some cleaning:

After some data exploration, we noticed that all the universities name are in the following format
> "{university_name} - {university_code}", e.g "Université de Genève - GE"

One of the university follow the same pattern but the hyphen (`-`) is missing, let's add it!

Then, we removed the hyphen and the `university_code` for all the universities.

In [5]:
def cleanup_names(row):
    if (row.University == 'Forschungskommission SAGW'):
        row.University = 'Forschungskommission - SAGW'
    # find the index in the string where the university name finishes
    end_of_name = row.University.rfind(' - ')
    # return the name of the university without the tag
    if end_of_name != -1:
        return row.University[:end_of_name]
    # print the name of the university if the format is wrong
    else:
        print("University format not expected : %s" % (row.University))
        return row.University

df_rare = df_blue.copy()
df_rare['University'] = df_rare.apply(cleanup_names, axis=1)

print(df_rare.shape)
df_rare.head(2)

(48283, 3)


,University,Approved Amount,Institution
1,Université de Genève,41022.0,Faculté de Psychologie et des Sciences de l'Ed...
2,"NPO (Biblioth., Museen, Verwalt.)",79732.0,Kommission für das Corpus philosophorum medii ...


## Localizing the Grants

In order to localize the universities, we first compute the list of all the universities and the list of all cantons, then we use the google map api to localize the universities.

In [6]:
universities = df_rare.University.unique()

print("number of universities: %d" % len(universities))

number of universities: 76


In [7]:
cantons_geo = r'ch-cantons.topojson.json'
with open(cantons_geo) as file:    
    topojson = json.load(file)
    cantons = [canton['id'] for canton in topojson['objects']['cantons']['geometries']]

print("number of cantons: %d" % len(cantons))

number of cantons: 26


In [8]:
def get_canton(name):
    query_params = {
        'address': name,
        'key': os.environ['GOOGLE_MAPS_API_KEY']
    }

    r = requests.get(
        'https://maps.googleapis.com/maps/api/geocode/json?',
        params=query_params
    )

    json_response = r.json()
    if json_response.get('status',None)=='OVER_QUERY_LIMIT':
        print('OVER_QUERY_LIMIT')
        return None
    
    for result in json_response['results']:
        for adress_component in result['address_components']:
            if 'administrative_area_level_1' in adress_component['types']:
                if adress_component['short_name'] in cantons:
                    return adress_component['short_name']
    return None

uni_canton = {}
for university in universities:
    uni_canton[university] = get_canton(university)

print("number of uni matched: %d" % len([0 for canton in uni_canton.values() if canton]))

number of uni matched: 27


We compute the amount included by our first search on the cantons. We obtain ~88% which pushes us to try to attribute cantons to more universities

In [9]:
def set_canton(row):
    return uni_canton[row.University]

df_medium = df_rare.copy()
df_medium['Canton'] = df_medium.apply(set_canton, axis=1)

print(df_medium.shape)
amount_included = df_medium[df_medium.apply(lambda row:not (row.Canton is None), axis=1)]['Approved Amount'].sum()
amount_not_included = df_medium[df_medium.apply(lambda row: row.Canton is None, axis=1)]['Approved Amount'].sum()
print("total approved amount included: %f CHF" % amount_included)
print("total approved amount not included: %f CHF" % amount_not_included)
print("percentage of amount included: %f" % (amount_included/amount_total))
df_medium.head(2)

(48283, 4)
total approved amount included: 11266984261.250004 CHF
total approved amount not included: 1568459215.770000 CHF
percentage of amount included: 0.877802


,University,Approved Amount,Institution,Canton
1,Université de Genève,41022.0,Faculté de Psychologie et des Sciences de l'Ed...,GE
2,"NPO (Biblioth., Museen, Verwalt.)",79732.0,Kommission für das Corpus philosophorum medii ...,None


In [10]:
uni_canton.update({
    'Université de Neuchâtel': 'NE',
    'Eidg. Anstalt für Wasserversorgung': 'ZH',
    'HES de Suisse occidentale': 'JU',
    'Zürcher Fachhochschule (ohne PH)': 'ZH',
    'Eidg. Material und Prüfungsanstalt': 'ZH',
    'Fachhochschule Nordwestschweiz (ohne PH)': 'ZH',
    'Inst. de Hautes Etudes Internat. et du Dév': 'GE',
    'Berner Fachhochschule': 'BE',
    'Idiap Research Institute': 'VS',
    'Friedrich Miescher Institute': 'BS'    
})
print("number of uni matched: %d" % len([0 for canton in uni_canton.values() if canton]))

df_medium['Canton'] = df_medium.apply(set_canton, axis=1)

print(df_medium.shape)
amount_included = df_medium[df_medium.apply(lambda row:not (row.Canton is None), axis=1)]['Approved Amount'].sum()
amount_not_included = df_medium[df_medium.apply(lambda row: row.Canton is None, axis=1)]['Approved Amount'].sum()
print("total approved amount included: %f CHF" % amount_included)
print("total approved amount not included: %f CHF" % amount_not_included)
print("percentage of amount included: %f" % (amount_included/amount_total))
df_medium.head(2)

number of uni matched: 37
(48283, 4)
total approved amount included: 12037466088.950006 CHF
total approved amount not included: 797977388.070000 CHF
percentage of amount included: 0.937830


,University,Approved Amount,Institution,Canton
1,Université de Genève,41022.0,Faculté de Psychologie et des Sciences de l'Ed...,GE
2,"NPO (Biblioth., Museen, Verwalt.)",79732.0,Kommission für das Corpus philosophorum medii ...,None


We can observe that two "Universities" have a lot of records and do not have a Canton, they are:
- NPO (Biblioth., Museen, Verwalt.) (corresponds to NGOs in different cantons)
- Firmen/Privatwirtschaft (corresponds to funding to private institutes in different cantons)

In order to assign a canton for these two "universities" and the remaining records, we will use the `Institution` field ! Let's manually fix some records with a `Institution` with a high value counts !

In [11]:
institutions = df_medium[df_medium.apply(
    lambda row: row.Canton is None,
    axis=1)].Institution.unique()

print("number of institutions: %d" % len(institutions))
institutions[:5]

number of institutions: 921


array(['Kommission für das Corpus philosophorum medii aevi der SGG',
       'Schweiz. Thesauruskommission',
       'Schweizerische Rechtsquellen c/o Universität Zürich / RWI',
       'Schweizerische Gesellschaft für Volkskunde',
       'Kuratorium Carl J. Burckhardt'], dtype=object)

In [14]:
# /!\ takes a lot of time /!\
ins_canton = {}
for institution in institutions:
    ins_canton[institution] = get_canton(institution)

print("number of institutions matched: %d" % len([0 for canton in ins_canton.values() if canton]))

number of institutions matched: 161


In [15]:
# manually adds a few cantons
ins_canton.update({
    'WSL - Institut für Schnee- und Lawinenforschung SLF': 'GR',
    'Forschungsanstalt Agroscope Reckenholz-Tänikon ART': 'ZH',
    'Schweizerisches Institut für Allergie- und Asthmaforschung': 'GR',
    'Institut universitaire romand de Santé au Travail': 'VD',
    "Muséum d'Histoire Naturelle": 'GE',
    'FORS c/o Université de Lausanne': 'VD',
    'Gesellschaft für Schweizerische Kunstgeschichte': 'BE'
})

print("number of institutions matched: %d" % len([0 for canton in ins_canton.values() if canton]))

number of institutions matched: 167


In [16]:
def set_canton_with_ins(row):
    return row.Canton if row.Canton else ins_canton.get(row.Institution,None)

df_medium['Canton'] = df_medium.apply(set_canton_with_ins, axis=1)

print(df_medium.shape)
amount_included = df_medium[df_medium.apply(lambda row:not (row.Canton is None), axis=1)]['Approved Amount'].sum()
amount_not_included = df_medium[df_medium.apply(lambda row: row.Canton is None, axis=1)]['Approved Amount'].sum()
print("total approved amount included: %f CHF" % amount_included)
print("total approved amount not included: %f CHF" % amount_not_included)
print("percentage of amount included: %f" % (amount_included/amount_total))
df_medium.head(2)

(48283, 4)
total approved amount included: 12247129507.530006 CHF
total approved amount not included: 588313969.490000 CHF
percentage of amount included: 0.954165


,University,Approved Amount,Institution,Canton
1,Université de Genève,41022.0,Faculté de Psychologie et des Sciences de l'Ed...,GE
2,"NPO (Biblioth., Museen, Verwalt.)",79732.0,Kommission für das Corpus philosophorum medii ...,None


## Visualization with Folium

We first prepare the dataset for the map. We simply need the `Approved Amount` summed and grouped by `Canton`. Folium expects a value for each canton, so we add rows for the missing cantons with `0` as `Approved Amount`. We add a new column with the `Approved Amount` value in millions.

In [17]:
df_well_done = df_medium[~df_medium.Canton.isnull()]
df_well_done = df_well_done[['Canton', 'Approved Amount']].groupby('Canton').sum().reset_index()

for canton in cantons:
    if not canton in df_well_done.Canton.tolist():
        df_well_done = df_well_done.append({
            'Canton': canton,
            'Approved Amount': 0
        }, ignore_index=True)

df_well_done['GEO_ID'] = df_well_done['Canton']
df_well_done['Approved Amount M'] = df_well_done['Approved Amount'] / 1e+06        
df_well_done = df_well_done[['GEO_ID', 'Approved Amount M']]

print(df_well_done.shape)
df_well_done.sort_values(by="Approved Amount M", ascending=False)

(26, 2)


,GEO_ID,Approved Amount M
20,ZH,3722.949255
17,VD,2405.243280
5,GE,1898.304781
1,BE,1581.481228
3,BS,1409.124528
4,FR,460.436496
9,NE,386.152704
0,AG,120.294504
11,SG,91.992498
8,LU,54.584378


In [18]:
grants_map = folium.Map(location=[46.85, 8.05], zoom_start=8)
grants_map.choropleth(
    geo_path=cantons_geo, 
    data=df_well_done,
    columns=['GEO_ID', 'Approved Amount M'],
    key_on='feature.id',
    topojson='objects.cantons',
    fill_color='YlOrRd',
    caption='Approved Amount',
    threshold_scale=[1, 10, 50, 500, 1000, 2000]
)
grants_map

TypeError: choropleth() got an unexpected keyword argument 'caption'